In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os, sys
import copy

In [ ]:
# If you have clone the repository and open this notebook,
# this notebook should be in likelihood-implementation/notebooks
# Let's set the working directory to be likelihood-implementation

likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory: ', likelihood_path)

### Check if running on correct environment

In [ ]:
print (os.environ['CONDA_DEFAULT_ENV'])

In [ ]:
# Import external loglike from the Likelihood Package within cobaya_interface.py

from cloe.cobaya_interface import EuclidLikelihood

In [ ]:
# Matplotlib params set-up

%matplotlib inline
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('font',size=25)
plt.rc('axes', titlesize=26)
plt.rc('axes', labelsize=25)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')

In [ ]:
# We are running the Euclid-Likelihood as an external likelihood class for Cobaya
# Cobaya needs a dictionary or yaml file as input to start running
# This dictionary below ('info') can be modified up to some point by the user to
# adapt it to the user's needs.
# The options that can be modified by the user are pointed with the acronym (UC).

info = {
    #'params': Cobaya's protected key of the input dictionary.
    # Includes the parameters that the user would like to sample over:
'params': {
        # (UC): each parameter below (which is a 'key' of another sub-dictionary) can contain a dictionary
        # with the key 'prior', 'latex'...
        # If the prior dictionary is not passed to a parameter, this parameter is fixed.
        # In this example, we are sampling the parameter ns
        # For more information see: https://cobaya.readthedocs.io/en/latest/example.html
        'ombh2': 0.022445, #Omega density of baryons times the reduced Hubble parameter squared
        'omch2': 0.1205579307, #Omega density of cold dark matter times the reduced Hubble parameter squared
        'H0': 67, #Hubble parameter evaluated today (z=0) in km/s/Mpc
        'tau': 0.0925, #optical depth
        'mnu': 0.06, #  sum of the mass of neutrinos in eV
        'nnu': 3.046, #N_eff of relativistic species
        'As': 2.12605e-9, #Amplitude of the primordial scalar power spectrum
        'ns': 0.96, # primordial power spectrum tilt
        'w': -1, #Dark energy fluid model
        'wa': 0, #Dark energy fluid model
        'omk': 0.0, #curvature density
        'omegam': None, #DERIVED parameter: Omega matter density
        'omegab': None, #DERIVED parameter: Omega baryon density
        'omeganu': None, #DERIVED parameter: Omega neutrino density
        'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
        'omegac': None, #DERIVED parameter: Omega cold dark matter density
        'N_eff': None,
        # (UC): baryonic feedback for built-in models (Mead2016/Mead2020_feedback; see next cell)
        # These parameters are considered only when the corresponding nonlinear modelling option is selected:
        # HMCode_A_baryon: 3.13 # default value for Mead2020
        # HMCode_eta_baryon: 0.603 # default value for Mead2020
        # HMCode_logT_AGN: 7.8 # default value for Mead2020_feedback
        # (UC): galaxy bias parameters:
        # The bias parameters below are currently fixed to the
        # values used by the Inter Science Taskforce: Forecast (IST:F)
        # and presented in the corresponding IST:F paper (arXiv: 1910.09273).
        # However, they can be changed by the user and even sample over them by putting a prior
        ############################
        # Spectroscopic parameters #
        ############################
        # Linear bias #
        'b1_spectro_bin1': 1.46, 'b1_spectro_bin2': 1.61, 'b1_spectro_bin3': 1.75, 'b1_spectro_bin4': 1.90,
        # Local quadratic bias #
        'b2_spectro_bin1': 0.0, 'b2_spectro_bin2': 0.0, 'b2_spectro_bin3': 0.0, 'b2_spectro_bin4': 0.0,
        # Non-local quadratic bias #
        'bG2_spectro_bin1': 0.0, 'bG2_spectro_bin2': 0.0, 'bG2_spectro_bin3': 0.0, 'bG2_spectro_bin4': 0.0,
        # Non-local quadratic bias #
        'bG3_spectro_bin1': 0.0, 'bG3_spectro_bin2': 0.0, 'bG3_spectro_bin3': 0.0, 'bG3_spectro_bin4': 0.0,
        # EFT counterterms (k^2) #
        'c0_spectro_bin1': 0.0, 'c0_spectro_bin2': 0.0, 'c0_spectro_bin3': 0.0, 'c0_spectro_bin4': 0.0,
        'c2_spectro_bin1': 0.0, 'c2_spectro_bin2': 0.0, 'c2_spectro_bin3': 0.0, 'c2_spectro_bin4': 0.0,
        'c4_spectro_bin1': 0.0, 'c4_spectro_bin2': 0.0, 'c4_spectro_bin3': 0.0, 'c4_spectro_bin4': 0.0,
        # EFT counterterms (k^4) #
        'ck4_spectro_bin1': 0.0, 'ck4_spectro_bin2': 0.0, 'ck4_spectro_bin3': 0.0, 'ck4_spectro_bin4': 0.0,
        # Deviations from Poisson shot-noise (constant) #
        'aP_spectro_bin1': 0.0, 'aP_spectro_bin2': 0.0, 'aP_spectro_bin3': 0.0, 'aP_spectro_bin4': 0.0,
        # Deviations from Poisson shot-noise (k^2) #
        'e0k2_spectro_bin1': 0.0, 'e0k2_spectro_bin2': 0.0, 'e0k2_spectro_bin3': 0.0, 'e0k2_spectro_bin4': 0.0,
        # Deviations from Poisson shot-noise (k^2 mu^2) #
        'e2k2_spectro_bin1': 0.0, 'e2k2_spectro_bin2': 0.0, 'e2k2_spectro_bin3': 0.0, 'e2k2_spectro_bin4': 0.0,
        # Inverse number density #
        'Psn_spectro_bin1': 0.0, 'Psn_spectro_bin2': 0.0, 'Psn_spectro_bin3': 0.0, 'Psn_spectro_bin4': 0.0,
        ##########################
        # Photometric parameters #
        ##########################
        # Linear photometric galaxy bias (IST:F case)
        'b1_photo_bin1': 1.0997727037892875,
        'b1_photo_bin2': 1.220245876862528,
        'b1_photo_bin3': 1.2723993083933989,
        'b1_photo_bin4': 1.316624471897739,
        'b1_photo_bin5': 1.35812370570578,
        'b1_photo_bin6': 1.3998214171814918,
        'b1_photo_bin7': 1.4446452851824907,
        'b1_photo_bin8': 1.4964959071110084,
        'b1_photo_bin9': 1.5652475842498528,
        'b1_photo_bin10': 1.7429859437184225,
        # Adding these here just for generation of params.yaml
        'b1_photo_bin11': 1.7429859437184225,
        'b1_photo_bin12': 1.7429859437184225,
        'b1_photo_bin13': 1.7429859437184225,
        # These values are the defaults for 13 bins
        # 'b1_photo_bin1': 1.03047,
        # 'b1_photo_bin2': 1.06699,
        # 'b1_photo_bin3': 1.17363,
        # 'b1_photo_bin4': 1.23340,
        # 'b1_photo_bin5': 1.27510,
        # 'b1_photo_bin6': 1.28574,
        # 'b1_photo_bin7': 1.39434,
        # 'b1_photo_bin8': 1.49170,
        # 'b1_photo_bin9': 1.52334,
        # 'b1_photo_bin10': 1.54639,
        # 'b1_photo_bin11': 1.68682,
        # 'b1_photo_bin12': 2.03066,
        # 'b1_photo_bin13': 2.57812,
        # Linear photometric galaxy bias (polynomial case)
        'b1_0_poly_photo': 0.830703,
        'b1_1_poly_photo': 1.190547,
        'b1_2_poly_photo': -0.928357,
        'b1_3_poly_photo': 0.423292,
        # Quadratic photometric galaxy bias
        'b2_photo_bin1': 0.0,
        'b2_photo_bin2': 0.0,
        'b2_photo_bin3': 0.0,
        'b2_photo_bin4': 0.0,
        'b2_photo_bin5': 0.0,
        'b2_photo_bin6': 0.0,
        'b2_photo_bin7': 0.0,
        'b2_photo_bin8': 0.0,
        'b2_photo_bin9': 0.0,
        'b2_photo_bin10': 0.0,
        'b2_photo_bin11': 0.0,
        'b2_photo_bin12': 0.0,
        'b2_photo_bin13': 0.0,
        # Quadratic photometric galaxy bias (polynomial case)
        'b2_0_poly_photo': 0.0,
        'b2_1_poly_photo': 0.0,
        'b2_2_poly_photo': 0.0,
        'b2_3_poly_photo': 0.0,
        # Non-local photometric galaxy biases
        'bG2_photo_bin1': 0.0, 'bG3_photo_bin1': 0.0,
        'bG2_photo_bin2': 0.0, 'bG3_photo_bin2': 0.0,
        'bG2_photo_bin3': 0.0, 'bG3_photo_bin3': 0.0,
        'bG2_photo_bin4': 0.0, 'bG3_photo_bin4': 0.0,
        'bG2_photo_bin5': 0.0, 'bG3_photo_bin5': 0.0,
        'bG2_photo_bin6': 0.0, 'bG3_photo_bin6': 0.0,
        'bG2_photo_bin7': 0.0, 'bG3_photo_bin7': 0.0,
        'bG2_photo_bin8': 0.0, 'bG3_photo_bin8': 0.0,
        'bG2_photo_bin9': 0.0, 'bG3_photo_bin9': 0.0,
        'bG2_photo_bin10': 0.0, 'bG3_photo_bin10': 0.0,
        'bG2_photo_bin11': 0.0, 'bG3_photo_bin11': 0.0,
        'bG2_photo_bin12': 0.0, 'bG3_photo_bin12': 0.0,
        'bG2_photo_bin13': 0.0, 'bG3_photo_bin13': 0.0,
        # Non-local photometric galaxy bias (polynomial case)
        'bG2_0_poly_photo': 0.0,
        'bG2_1_poly_photo': 0.0,
        'bG2_2_poly_photo': 0.0,
        'bG2_3_poly_photo': 0.0,
        'bG3_0_poly_photo': 0.0,
        'bG3_1_poly_photo': 0.0,
        'bG3_2_poly_photo': 0.0,
        'bG3_3_poly_photo': 0.0,
        # Magnification bias
        'magnification_bias_1': -0.8499,
        'magnification_bias_2': -1.023,
        'magnification_bias_3': -0.9474,
        'magnification_bias_4': -0.9837,
        'magnification_bias_5': -0.7699,
        'magnification_bias_6': -0.7137,
        'magnification_bias_7': -0.52345,
        'magnification_bias_8': -0.40845,
        'magnification_bias_9': -0.41045,
        'magnification_bias_10': -0.0824,
        'magnification_bias_11': 0.4917,
        'magnification_bias_12': 0.98555,
        'magnification_bias_13': 1.6056,
        'b0_mag_poly' : 0.0,
        'b1_mag_poly' : 0.0,
        'b2_mag_poly' : 0.0,
        'b3_mag_poly' : 0.0,
        # Multiplicative bias
        'multiplicative_bias_1': 0.0,
        'multiplicative_bias_2': 0.0,
        'multiplicative_bias_3': 0.0,
        'multiplicative_bias_4': 0.0,
        'multiplicative_bias_5': 0.0,
        'multiplicative_bias_6': 0.0,
        'multiplicative_bias_7': 0.0,
        'multiplicative_bias_8': 0.0,
        'multiplicative_bias_9': 0.0,
        'multiplicative_bias_10': 0.0,
        'multiplicative_bias_11': 0.0,
        'multiplicative_bias_12': 0.0,
        'multiplicative_bias_13': 0.0,
        ###############################################################################
        # Baryonic parameters
        # Two emulators have different parameters and different ways of setting them:
        #  - Parameters can be set for each redshift bin as par_i, with i= 1, ..., 13  (currently max 13 bins)
        #  - Or as a power law in redshift par(z) = par_0 * (1+z)^(-nu_par)
        ###############################################################################
        # Example BCemu baryon params (from fit to BAHAMAS for 10 bins, values for bins 11 - 13 are the same as bin 10)
        'log10Mc_bcemu_bin1': 13.402342835406102, 'log10Mc_bcemu_bin2': 13.255603572838046, 'log10Mc_bcemu_bin3': 13.39534479993441,
        'log10Mc_bcemu_bin4': 13.491189798802624, 'log10Mc_bcemu_bin5': 13.517099731729203, 'log10Mc_bcemu_bin6': 13.431948448938623,
        'log10Mc_bcemu_bin7': 13.159142065726643, 'log10Mc_bcemu_bin8': 12.630530198678242, 'log10Mc_bcemu_bin9': 11.768928839012311,
        'log10Mc_bcemu_bin10': 11.123969034839764,'log10Mc_bcemu_bin11': 11.123969034839764,'log10Mc_bcemu_bin12': 11.123969034839764,
        'log10Mc_bcemu_bin13': 11.123969034839764,
        'mu_bcemu_bin1': 1.081169575215787, 'mu_bcemu_bin2': 1.0748796706532235, 'mu_bcemu_bin3': 1.257505674386812,
        'mu_bcemu_bin4': 1.3953104854720346, 'mu_bcemu_bin5': 1.4707848100185459, 'mu_bcemu_bin6': 1.451326233210004,
        'mu_bcemu_bin7': 1.2738670043792995, 'mu_bcemu_bin8': 0.8851923901330802, 'mu_bcemu_bin9': 0.23572246186875243,
        'mu_bcemu_bin10': -0.006953332782079408, 'mu_bcemu_bin11': -0.006953332782079408, 'mu_bcemu_bin12': -0.006953332782079408,
        'mu_bcemu_bin13': -0.006953332782079408,
        'thej_bcemu_bin1': 3.6711008267835776, 'thej_bcemu_bin2': 4.0853297524846175, 'thej_bcemu_bin3': 4.1001870114090835,
        'thej_bcemu_bin4': 4.085897535238355, 'thej_bcemu_bin5': 4.062128106370927, 'thej_bcemu_bin6': 4.0416849190138695,
        'thej_bcemu_bin7': 4.0405617133891685, 'thej_bcemu_bin8': 4.073797196972992, 'thej_bcemu_bin9': 4.168188042786498,
        'thej_bcemu_bin10': 4.567495847826163, 'thej_bcemu_bin11': 4.567495847826163, 'thej_bcemu_bin12': 4.567495847826163,
        'thej_bcemu_bin13': 4.567495847826163,
        'gamma_bcemu_bin1': 2.732738146480909, 'gamma_bcemu_bin2': 2.794658925091356, 'gamma_bcemu_bin3': 3.008141783989797,
        'gamma_bcemu_bin4': 3.2186226058208787, 'gamma_bcemu_bin5': 3.425034377127418, 'gamma_bcemu_bin6': 3.6237277892257937,
        'gamma_bcemu_bin7': 3.805233587893605, 'gamma_bcemu_bin8': 3.952453083293017, 'gamma_bcemu_bin9': 4.012632091622541,
        'gamma_bcemu_bin10': 3.1454520255604006, 'gamma_bcemu_bin11': 3.1454520255604006, 'gamma_bcemu_bin12': 3.1454520255604006,
        'gamma_bcemu_bin13': 3.1454520255604006,
        'delta_bcemu_bin1': 6.407442578598766, 'delta_bcemu_bin2': 7.0846518716943025, 'delta_bcemu_bin3': 7.226700228338714,
        'delta_bcemu_bin4': 7.343194626402527, 'delta_bcemu_bin5': 7.470370166898234, 'delta_bcemu_bin6': 7.638743359373677,
        'delta_bcemu_bin7': 7.891612514615537, 'delta_bcemu_bin8': 8.252389998618296, 'delta_bcemu_bin9': 8.702184867164187,
        'delta_bcemu_bin10': 7.994110032082859, 'delta_bcemu_bin11': 7.994110032082859, 'delta_bcemu_bin12': 7.994110032082859,
        'delta_bcemu_bin13': 7.994110032082859,
        'eta_bcemu_bin1': 0.20680675958346154, 'eta_bcemu_bin2': 0.22813237994091348, 'eta_bcemu_bin3': 0.21989900653826788,
        'eta_bcemu_bin4': 0.2090499814223423, 'eta_bcemu_bin5': 0.19645554090162448, 'eta_bcemu_bin6': 0.18238365514915927,
        'eta_bcemu_bin7': 0.16686344766458094, 'eta_bcemu_bin8': 0.14883503959461816, 'eta_bcemu_bin9': 0.1240089652119986,
        'eta_bcemu_bin10': 0.044931761090432315, 'eta_bcemu_bin11': 0.044931761090432315, 'eta_bcemu_bin12': 0.044931761090432315,
        'eta_bcemu_bin13': 0.044931761090432315,
        'deta_bcemu_bin1': 0.08189803795251066, 'deta_bcemu_bin2': 0.05028436917683151, 'deta_bcemu_bin3': 0.04842000359519622,
        'deta_bcemu_bin4': 0.04869423933122641, 'deta_bcemu_bin5': 0.04959595282843388, 'deta_bcemu_bin6': 0.05011878848954406,
        'deta_bcemu_bin7': 0.04907477885822071, 'deta_bcemu_bin8': 0.04681178740273659, 'deta_bcemu_bin9': 0.048062862071344824,
        'deta_bcemu_bin10': 0.14550946518064195, 'deta_bcemu_bin11': 0.14550946518064195, 'deta_bcemu_bin12': 0.14550946518064195,
        'deta_bcemu_bin13': 0.14550946518064195,
        # BCemu params for the redshift parametrization
        # To mimic the BCemu 3-parameter model, just vary the first 6 params and leave the others fixed to these values.
        # A 1-parameter model would use just log10Mc (and others fixed to current values)
        'log10Mc_bcemu_0': 13.32,
        'nu_log10Mc_bcemu': -0.15,
        'thej_bcemu_0': 3.5,
        'nu_thej_bcemu': 0.0,
        'deta_bcemu_0': 0.2,
        'nu_deta_bcemu': 0.6,
        'mu_bcemu_0': 1.0,
        'nu_mu_bcemu': 0.0,
        'gamma_bcemu_0': 2.5,
        'nu_gamma_bcemu': 0.0,
        'delta_bcemu_0': 7.0,
        'nu_delta_bcemu': 0.0,
        'eta_bcemu_0': 0.2,
        'nu_eta_bcemu': 0.0,
        # Example Bacco baryon parameters (from fit to BAHAMAS for 10 bins, values for bins 11 - 13 are the same as bin 10)
        'M_c_bacco_bin1': 14.158079090897681,'M_c_bacco_bin2': 14.413536974582364,
        'M_c_bacco_bin3': 14.513655381066439,
        'M_c_bacco_bin4': 14.59501140993621,
        'M_c_bacco_bin5': 14.667602932620955,
        'M_c_bacco_bin6': 14.736117785720888,
        'M_c_bacco_bin7': 14.803902874910724,
        'M_c_bacco_bin8': 14.873354165369339,
        'M_c_bacco_bin9': 14.947678648592403,
        'M_c_bacco_bin10': 15.179943239223197,
        'M_c_bacco_bin11': 15.179943239223197,
        'M_c_bacco_bin12': 15.179943239223197,
        'M_c_bacco_bin13': 15.179943239223197,
        'eta_bacco_bin1': -0.34397323435383403, 'eta_bacco_bin2': -0.3823001659654948,
        'eta_bacco_bin3': -0.3666759403458338,
        'eta_bacco_bin4': -0.34766555599217897,
        'eta_bacco_bin5': -0.32723937222645033,
        'eta_bacco_bin6': -0.3066851485202732,
        'eta_bacco_bin7': -0.28793191687619096,
        'eta_bacco_bin8': -0.2761849439738153,
        'eta_bacco_bin9': -0.29044476508897765,
        'eta_bacco_bin10': -0.3012527587604943,
        'eta_bacco_bin11': -0.3012527587604943,
        'eta_bacco_bin12': -0.3012527587604943,
        'eta_bacco_bin13': -0.3012527587604943,
        'beta_bacco_bin1': -0.1401597529600361,'beta_bacco_bin2': -0.14602731261184787,
        'beta_bacco_bin3': -0.15945179021822595,
        'beta_bacco_bin4': -0.1717165402260191,
        'beta_bacco_bin5': -0.18260340867450087,
        'beta_bacco_bin6': -0.1914516575772147,
        'beta_bacco_bin7': -0.19665359534532956,
        'beta_bacco_bin8': -0.1938633002915493,
        'beta_bacco_bin9': -0.16799956045034847,
        'beta_bacco_bin10': -0.1180328507054961,
        'beta_bacco_bin11': -0.1180328507054961,
        'beta_bacco_bin12': -0.1180328507054961,
        'beta_bacco_bin13': -0.1180328507054961,
        'M1_z0_cen_bacco_bin1': 11.340025775544985,'M1_z0_cen_bacco_bin2': 11.30961906881156,
        'M1_z0_cen_bacco_bin3': 11.366002025818862,
        'M1_z0_cen_bacco_bin4': 11.423530231429538,
        'M1_z0_cen_bacco_bin5': 11.479266623369345,
        'M1_z0_cen_bacco_bin6': 11.529947744472405,
        'M1_z0_cen_bacco_bin7': 11.569052829475833,
        'M1_z0_cen_bacco_bin8': 11.578833725581436,
        'M1_z0_cen_bacco_bin9': 11.496367669453079,
        'M1_z0_cen_bacco_bin10': 11.360811343412026,
        'M1_z0_cen_bacco_bin11': 11.360811343412026,
        'M1_z0_cen_bacco_bin12': 11.360811343412026,
        'M1_z0_cen_bacco_bin13': 11.360811343412026,
        'theta_inn_bacco_bin1': -0.9790217925155928, 'theta_inn_bacco_bin2': -1.3854647301685985,
        'theta_inn_bacco_bin3': -1.3604452515853098,
        'theta_inn_bacco_bin4': -1.2949667258085438,
        'theta_inn_bacco_bin5': -1.2056425079533284,
        'theta_inn_bacco_bin6': -1.0991845011332575,
        'theta_inn_bacco_bin7': -0.9805620838281162,
        'theta_inn_bacco_bin8': -0.8624918408856748,
        'theta_inn_bacco_bin9': -0.7968588249755786,
        'theta_inn_bacco_bin10': -0.5312787510657424,
        'theta_inn_bacco_bin11': -0.5312787510657424,
        'theta_inn_bacco_bin12': -0.5312787510657424,
        'theta_inn_bacco_bin13': -0.5312787510657424,
        'M_inn_bacco_bin1': 12.472,'M_inn_bacco_bin2': 11.098133175636903,
        'M_inn_bacco_bin3': 10.685545014634902,
        'M_inn_bacco_bin4': 10.379401082211185,
        'M_inn_bacco_bin5': 10.124549632511826,
        'M_inn_bacco_bin6': 9.89485323082124,
        'M_inn_bacco_bin7': 9.669674533440968,
        'M_inn_bacco_bin8': 9.423522683043274,
        'M_inn_bacco_bin9': 9.086255671995564,
        'M_inn_bacco_bin10': 8.119132619254136,
        'M_inn_bacco_bin11': 8.119132619254136,
        'M_inn_bacco_bin12': 8.119132619254136,
        'M_inn_bacco_bin13': 8.119132619254136,
        'theta_out_bacco_bin1': 0.2672123790126238,'theta_out_bacco_bin2': 0.21845633941784046,
        'theta_out_bacco_bin3': 0.22229391886311567,
        'theta_out_bacco_bin4': 0.23124624733764712,
        'theta_out_bacco_bin5': 0.24338032280596764,
        'theta_out_bacco_bin6': 0.25802109222979985,
        'theta_out_bacco_bin7': 0.27483978056155306,
        'theta_out_bacco_bin8': 0.29287229590732955,
        'theta_out_bacco_bin9': 0.3076030069491944,
        'theta_out_bacco_bin10': 0.3569278707998519,
        'theta_out_bacco_bin11': 0.3569278707998519,
        'theta_out_bacco_bin12': 0.3569278707998519,
        'theta_out_bacco_bin13': 0.3569278707998519,
        # Bacco params for the redshift parametrization
        # Here too, one can use just the first 6 params to get a simpler model, with the others fixed.
        # Note that the redshift dependence has not been tested by the Bacco collaboration.
        'M_c_bacco_0': 13.25,
        'nu_M_c_bacco': -0.15,
        'eta_bacco_0': -0.3,
        'nu_eta_bacco': 0.0,
        'beta_bacco_0': -0.22,
        'nu_beta_bacco': 0.0,
        'M1_z0_cen_bacco_0': 10.5,
        'nu_M1_z0_cen_bacco': 0.0,
        'theta_inn_bacco_0': -0.86,
        'nu_theta_inn_bacco': 0.0,
        'M_inn_bacco_0': 13.4,
        'nu_M_inn_bacco': 0.0,
        'theta_out_bacco_0': 0.25,
        'nu_theta_out_bacco': 0.0,
        # Intrinsic alignment parameters
        'a1_ia': 1.72,
        'a2_ia': 2,
        'b1_ia': 1,
        'eta1_ia': -0.41,
        'eta2_ia': 1,
        'beta1_ia': 0.0,
        'pivot_redshift': 0.0,
        # Redshift distributions nuisance parameters: shifts
        'dz_1_GCphot': 0., 'dz_1_WL': 0.,
        'dz_2_GCphot': 0., 'dz_2_WL': 0.,
        'dz_3_GCphot': 0., 'dz_3_WL': 0.,
        'dz_4_GCphot': 0., 'dz_4_WL': 0.,
        'dz_5_GCphot': 0., 'dz_5_WL': 0.,
        'dz_6_GCphot': 0., 'dz_6_WL': 0.,
        'dz_7_GCphot': 0., 'dz_7_WL': 0.,
        'dz_8_GCphot': 0., 'dz_8_WL': 0.,
        'dz_9_GCphot': 0., 'dz_9_WL': 0.,
        'dz_10_GCphot': 0., 'dz_10_WL': 0.,
        'dz_11_GCphot': 0., 'dz_11_WL': 0.,
        'dz_12_GCphot': 0., 'dz_12_WL': 0.,
        'dz_13_GCphot': 0., 'dz_13_WL': 0.,
        # Growth index
        'gamma_MG': 0.55,
        'f_out': 0.0,
        'f_out_1': 0.0,
        'f_out_2': 0.0,
        'f_out_3': 0.0,
        'f_out_4': 0.0,
        'sigma_z': 0.002},
    #'theory': Cobaya's protected key of the input dictionary.
    # Cobaya needs to ask some minimum theoretical requirements to a Boltzman Solver
    # (UC): you can choose between CAMB or CLASS
    # In this DEMO, we use CAMB and specify some CAMB arguments
    # such as the number of massive neutrinos
    # and the dark energy model
    #
    # ATTENTION: If you have CAMB/CLASS already installed and
    # you are not using the likelihood conda environment
    # or option (2) in cell (3) (Cobaya modules), you can add an extra key called 'path' within the camb dictionary
    # to point to your already installed CAMB code
    # NOTE: for values of the nonlinear flag larger than 0, a new key is added in info['theory']['camb']['extra_args'],
    # i.e. 'halofit_version', which contains the requested version of halofit, as described above
    'theory': {'camb':
               {'stop_at_error': True,
                'extra_args':{'num_massive_neutrinos': 1,
                              'dark_energy_model': 'ppf'}}},
    #'sampler': Cobaya's protected key of the input dictionary.
    # (UC): you can choose the sampler you want to use.
    # Check Cobaya's documentation to see the list of available samplers
    # In this DEMO, we use the 'evaluate' sampler to make a single computation of the posterior distributions
    # Note: if you want to run a simple MCMC sampling choose 'mcmc'
    'sampler': {'evaluate': None},
    # 'packages_path': Cobaya's protected key of the input dictionary.
    # This is the variable you need to update
    # if you are running Cobaya with cobaya_modules (option (2) above).
    # If you are using the conda likelihood environment or option (1),
    # please, keep the line below commented
    #
    #'packages_path': modules_path,
    #
    #'output': Cobaya's protected key of the input dictionary.
    # Where are the results going to be stored, in case that the sampler produce output files?
    # For example: chains...
    # (UC): modify the path below within 'output' to choose a name and a directory for those files
    'output': 'chains/my_euclid_experiment',
    #'debug': Cobaya's protected key of the input dictionary.
    # (UC): how much information you want Cobaya to print? If debug: True, it prints every single detail
    # that is going on internally in Cobaya
    'debug': False,
    #'timing': Cobaya's protected key of the input dictionary.
    # (UC): if timing: True, Cobaya returns how much time it took it to make a computation of the posterior
    # and how much time take each of the modules to perform their tasks
    'timing': True,
    #'force': Cobaya's protected key of the input dictionary.
    # (UC): if 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    'force': True,
    }

In [ ]:
#'likelihood': Cobaya's protected key of the input dictionary.
# (UC): The user can select which data wants to use for the analysis.
# Check Cobaya's documentation to see the list of the current available data experiments
# In this DEMO, we load the Euclid-Likelihood as an external function, and name it 'Euclid'
info['likelihood'] = {'Euclid':
                     {'external': EuclidLikelihood, # Likelihood Class to be read as external
                     # Note: everything down below will overwrite the information read
                     # in the config folder
                     #
                     # Select which observables want to use during the analysis
                     'observables_selection': {
                         'WL': {'WL': True, 'GCphot': False, 'GCspectro': False},
                         'GCphot': {'GCphot': True, 'GCspectro': False},
                         'GCspectro': {'GCspectro': False},
                         'CG':{'CG': False},
                         'add_phot_RSD': False,
                         'matrix_transform_phot' : False,
                     },
                     # Plot the selected observables matrx
                     'plot_observables_selection': False,
                     'solver': 'camb',
                     # Nonlinear flags
                     # With this, the user can specify which nonlinear model they want
                     # For the time-being the available options are:
                     # NL_flag_phot_matter
                        # 0 -> linear-only (from Cosmology class)
                        # 1 -> Takahashi
                        # 2 -> Mead2016 (w/o baryon corrections)
                        # 3 -> Mead2020 (w/o baryon corrections)
                        # 4 -> Euclid Emulator 2
                        # 5 -> Bacco (matter)
                     'NL_flag_phot_matter': 3,
                     # NL_flag_phot_bias
                        # 0 -> linear only
                        # 1 -> Non-linear PT
                     'NL_flag_phot_bias': 1,
                     'bias_model': 1,
                     'NL_flag_phot_baryon': 0,
                     # This flag sets the redshift evolution for baryonic parameters for emulators
                     # The options are:
                            # True -> use X(z) = X_0 * (1+z)^(-nu_X), no. of params: 7*2 = 14
                            # False -> use X_i at each redshift bin i and interpolate, no. of params: 7*10 = 70
                     'Baryon_redshift_model': False,
                     # IA_flag
                        # 0 -> NLA
                        # 1 -> TATT
                    'IA_flag':0,
                    'IR_resum': 'DST',
                    'NL_flag_spectro': 0,
                    'f_out_z_dep': False,
                    'GCsp_z_err' : False,
                     # Use MG gamma
                     'use_gamma_MG': False,
                     'use_Weyl': False,
                     #'data': This give specifications for the paths of the input data files
                     'data': {
                        #'sample' specifies the first folder below the main data folder
                        'sample': 'ExternalBenchmark',
                        #'spectro' and 'photo' specify paths to data files.
                        'spectro': {
                            # GC Spectro root name should contain z{:s} string
                            # to enable iteration over bins
                            'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                            'redshifts': ["1.", "1.2", "1.4", "1.65"],
                            'edges': [0.9, 1.1, 1.3, 1.5, 1.8]},
                        'photo': {
                            'redshifts': [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038],
                            'ndens_GC': 'niTab-EP10-RB00.dat',
                            'ndens_WL': 'niTab-EP10-RB00.dat',
                            'luminosity_ratio': 'luminosity_ratio.dat',
                            'Fourier': True,
                            'photo_data': 'standard',
                            # Photometric root names should contain z{:s} string
                            # to specify IA model
                            'root_GC': 'Cls_{:s}_PosPos.dat',
                            'root_WL': 'Cls_{:s}_ShearShear.dat',
                            'root_XC': 'Cls_{:s}_PosShear.dat',
                            'root_mixing_matrix': 'fs2_mms_10zbins_32ellbins.fits',
                            'IA_model': 'zNLA',
                            # Photometric covariances root names should contain z{:s} string
                            # to specify how the covariance was calculated
                            'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npz',
                            'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npz',
                            'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npz',
                            'cov_model': 'Gauss'}},

                    }}

In [ ]:
# We need to call this method to update the params.yaml file that contains all the nuisance parameters of CLOE
# that would be otherwise unkwown to COBAYA. Here the update is based on the info dictionary defined above
from cloe.auxiliary.likelihood_yaml_handler import write_params_yaml_from_info_dict
write_params_yaml_from_info_dict(info)

In [ ]:
from cobaya.model import get_model
# The `get_model` function of Cobaya imported in the line above needs a yaml or dictionary as an argument

# We are setting the nonlinear model for the matter power spectrum coming
# from the Boltzmann solver according to the value of the nonlinear flag
# for the photometric probes, NL_flag_photo_matter
from cloe.auxiliary.likelihood_yaml_handler import set_halofit_version
set_halofit_version(info, info['likelihood']['Euclid']['NL_flag_phot_matter'],
                          info['likelihood']['Euclid']['NL_flag_phot_baryon'])
# We measure the time to give us an estimation of how much time it takes to make the initialization of the
# likelihood
t1 = time.time()

# Create an instance of the `model` wrapper called model
model = get_model(info)
print('Time for initialization of the likelihood: ', time.time()-t1)
# We need to sample the log posterior to create model.provider.params
logposterior = model.logposterior({})

In [ ]:
# Create an instance of the class EuclidLikelihood and initialize it (general case)
like = EuclidLikelihood()
like.initialize()
like.passing_requirements(model, info, **model.provider.params)


### Copy the cosmo dictionary and produce new ones with photometric non-linear bias activated.
 - Case (1) : $b_2 = f(b_1)$ and $b_{\mathcal{G}_2} = b_{\Gamma_3} = 0$
 - Case (2) : $b_2 = f(b_1)$ and $b_{\mathcal{G}_2} = -2/7(b_1-1)$ and $b_{\Gamma_3} = -1/6(1-b_1) -5/2\ b_{\mathcal{G}_2}$

In [ ]:
# Function for b1-b2 relation
# Taken from Hoffman et al. 2016 : https://arxiv.org/pdf/1607.01024.pdf
def b2_of_b1(b1):
    #return -0.2*b1/b1
    return (b1**2 - b1*2.43 + 0.77)

In [ ]:
# This is just the scale at which to get the growth factor, it is used as an input for update_cosmo_dic
wavenumber_growth = 0.05

# Linear bias case
like.cosmo.cosmo_dic['NL_flag_phot_bias'] = 0
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic = copy.copy(like.cosmo.cosmo_dic)

# Non-linear bias case (1)
like = EuclidLikelihood()
like.initialize()
like.passing_requirements(model, info, **model.provider.params)
# Activate the non-linear bias option
like.cosmo.cosmo_dic['NL_flag_phot_bias'] = 1
# Set the values for b2 at the center of each tomorgaphic bins.
# The value at all redshift is then linearly interpolated.
nuisance_params_dic = like.cosmo.cosmo_dic['nuisance_parameters']
for i in range(1,14):
    nuisance_params_dic['b2_photo_bin'+str(i)] = b2_of_b1(nuisance_params_dic['b1_photo_bin'+str(i)])
# Update and copy dictionnary
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_nlbias_b2 = copy.copy(like.cosmo.cosmo_dic)

# Non-linear bias case (2)
like = EuclidLikelihood()
like.initialize()
like.passing_requirements(model, info, **model.provider.params)
# Activate the non-linear bias option
like.cosmo.cosmo_dic['NL_flag_phot_bias'] = 1
# Set the values for bG2 and bG3 at the center of each tomorgaphic bins.
# The value at all redshift is then linearly interpolated.
nuisance_params_dic = like.cosmo.cosmo_dic['nuisance_parameters']
for i in range(1,14):
    nuisance_params_dic['b2_photo_bin'+str(i)] = b2_of_b1(nuisance_params_dic['b1_photo_bin'+str(i)])
    nuisance_params_dic['bG2_photo_bin'+str(i)] = -(2/7.)*(nuisance_params_dic['b1_photo_bin'+str(i)]-1)
    nuisance_params_dic['bG3_photo_bin'+str(i)] = -(1/6)*(nuisance_params_dic['b1_photo_bin'+str(i)]-1) \
                                                    - (5/2)*nuisance_params_dic['bG2_photo_bin'+str(i)]
# Update and copy dictionnary
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], wavenumber_growth)
cosmo_dic_nlbias_nonlocal = copy.copy(like.cosmo.cosmo_dic)

### Plot $b_1$, $b_2$, $b_{\mathcal{G}_2}$ and $b_{\Gamma_3}$ redshift evolution

In [ ]:
z_win = like.cosmo.cosmo_dic['z_win']
fig, ax = plt.subplots(1,1)
ax.plot(z_win, cosmo_dic_nlbias_nonlocal['b1_inter'](z_win), label='$b_1$')
ax.plot(z_win, cosmo_dic_nlbias_nonlocal['b2_inter'](z_win), label='$b_2$')
ax.plot(z_win, cosmo_dic_nlbias_nonlocal['bG2_inter'](z_win), label='$b_{\mathcal{G}_2}$')
ax.plot(z_win, cosmo_dic_nlbias_nonlocal['bG3_inter'](z_win), label='$b_{\Gamma_3}$')
ax.set_xlabel('$z$')
ax.set_ylabel('Galaxy bias')
ax.legend(frameon=False, ncol=3)
plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## Choose wavenumber-array for plots
wavenumbers=np.logspace(-5, 1, 10000)

## Obtain set of defined redshift range
like.cosmo.cosmo_dic['z_win']

## Choose a set of z's to plot
redshift_chosen = [like.cosmo.cosmo_dic['z_win'][ii] for ii in [0,10,37,50]]

In [ ]:
from matplotlib.pyplot import cm
from matplotlib.cm import get_cmap

## Define a color palette for the plots
name = "Dark2"
cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
colors = cmap.colors

### Plot the Pgg_photo and Pgm_photo

In [ ]:
nonlinear_dic = like.cosmo.nonlinear.nonlinear_dic

fig, ax = plt.subplots(1, 1, figsize=(16,10))
fig.suptitle('Different contributions at $z=0$')

ax.semilogx(wavenumbers, cosmo_dic['Pmm_phot'](0, wavenumbers).flatten(), '--',
           label=r"Matter, Halofit $P_{NL}(k)$ ", c=colors[0])

ax.semilogx(wavenumbers, cosmo_dic_nlbias_nonlocal['Pgg_phot'](0, wavenumbers).flatten(), '-',
           label=r"$P_{gg}(k)$ with $(b_1, b_2, b_{\mathcal{G}_2}, b_{\Gamma_3})$ ", c=colors[1])

ax.semilogx(wavenumbers, nonlinear_dic['Pb1b2_kz'](0, wavenumbers).flatten(), ':',
           label=r"$P_{b_1,b_2}(k)$ ", c=colors[2])

ax.semilogx(wavenumbers, nonlinear_dic['Pb2b2_kz'](0, wavenumbers).flatten(), ':',
           label=r"$P_{b_2,b_2}(k)$ ", c=colors[3])

ax.semilogx(wavenumbers, nonlinear_dic['Pb1bG2_kz'](0, wavenumbers).flatten(), ':',
           label=r"$P_{b_1,b_{\mathcal{G}_2}}(k)$ ", c=colors[4])

ax.semilogx(wavenumbers, (nonlinear_dic['Pb2bG2_kz'](0, wavenumbers)+nonlinear_dic['PZ1bG2_kz'](0, wavenumbers)).flatten(), ':',
           label=r"$P_{b_2,b_{\mathcal{G}_2}}(k)$ ", c=colors[5])

ax.semilogx(wavenumbers, nonlinear_dic['PbG2bG2_kz'](0, wavenumbers).flatten(), ':',
           label=r"$P_{b_{\mathcal{G}_2},b_{\mathcal{G}_2}}(k)$ ", c=colors[6])

ax.semilogx(wavenumbers, nonlinear_dic['PZ1bG3_kz'](0, wavenumbers).flatten(), ':',
           label=r"$P_{Z1,b_{\Gamma_3}}(k)$ ", c=colors[0])

ax.set_xlabel(r'$k$ $[{Mpc}^{-1}]$', fontsize=24)
ax.set_ylabel(r'$P(k) [{Mpc}^{-3}]$')
ax.legend(frameon=False, fontsize=24, ncol=1, loc='upper left')#, bbox_to_anchor=(0,1.5))

plt.show()

In [ ]:
## Create four panels, one for each of the chosen redshifts above

fig, axs = plt.subplots(2, 2, figsize=(24,18))
fig.suptitle('Galaxy-Galaxy power spectrum')

ax=axs[0,0]

Pab = 'Pgg_phot'


for iz in range(4):
    if iz==0:
        ax=axs[0,0]
    if iz==1:
        ax=axs[0,1]
    if iz==2:
        ax=axs[1,0]
    if iz==3:
        ax=axs[1,1]
    redshift0=redshift_chosen[iz]

    ax.loglog(wavenumbers, cosmo_dic['Pmm_phot'](redshift0, wavenumbers).flatten(), ':',
               label=r"Matter $P_{NL}(k)$, HMcode2020, z="+str(round(redshift0,2)), c=colors[0])

    ax.loglog(wavenumbers, cosmo_dic[Pab](redshift0, wavenumbers).flatten(), '-',
               label=r"$P_{gg}(k)$ Linear bias z="+str(round(redshift0,2)), c=colors[1])

    ax.loglog(wavenumbers, cosmo_dic_nlbias_b2[Pab](redshift0, wavenumbers).flatten(), '-',
               label=r"$P_{gg}(k)$ Non-linear bias with $(b_1, b_2)$ z="+str(round(redshift0,2)), c=colors[2])

    ax.loglog(wavenumbers, cosmo_dic_nlbias_nonlocal[Pab](redshift0, wavenumbers).flatten(), '--',
               label=r"$P_{gg}(k)$ Non-linear bias with $(b_1, b_2, b_{\mathcal{G}_2}, b_{\Gamma_3})$ z="+str(round(redshift0,2)), c=colors[3])

    ax.set_xlabel(r'$k$ $[{Mpc}^{-1}]$', fontsize=24)
    ax.set_ylabel(r'$P(k) [{Mpc}^{-3}]$')
    ax.legend(frameon=False, fontsize=24)
plt.show()

In [ ]:
## Create four panels, one for each of the chosen redshifts above

fig, axs = plt.subplots(2, 2, figsize=(24,18))
fig.suptitle('Galaxy-Matter power spectrum')

ax=axs[0,0]

Pab = 'Pgdelta_phot'


for iz in range(4):
    if iz==0:
        ax=axs[0,0]
    if iz==1:
        ax=axs[0,1]
    if iz==2:
        ax=axs[1,0]
    if iz==3:
        ax=axs[1,1]
    redshift0=redshift_chosen[iz]

    ax.loglog(wavenumbers, cosmo_dic['Pmm_phot'](redshift0, wavenumbers).flatten(), ':',
               label=r"Matter $P_{NL}(k)$, Halofit, z="+str(round(redshift0,2)), c=colors[0])


    ax.loglog(wavenumbers, cosmo_dic[Pab](redshift0, wavenumbers).flatten(), '-',
               label=r"$P_{gm}(k)$ Linear bias z="+str(round(redshift0,2)), c=colors[1])

    ax.loglog(wavenumbers, cosmo_dic_nlbias_b2[Pab](redshift0, wavenumbers).flatten(), '-',
               label=r"$P_{gg}(k)$ Non-linear bias with $(b_1, b_2)$ z="+str(round(redshift0,2)), c=colors[2])

    ax.loglog(wavenumbers, cosmo_dic_nlbias_nonlocal[Pab](redshift0, wavenumbers).flatten(), '--',
               label=r"$P_{gg}(k)$ Non-linear bias with $(b_1, b_2, b_{\mathcal{G}_2}, b_{\Gamma_3})$ z="+str(round(redshift0,2)), c=colors[3])


    ax.set_xlabel(r'$k$ $[{Mpc}^{-1}]$', fontsize=24)
    ax.set_ylabel(r'$P(k) [{Mpc}^{-3}]$')
    ax.legend(frameon=False, fontsize=24)
plt.show()

### Plotting the C_ells manually using the Photo class and data Reader

In [ ]:
from cloe.photometric_survey.photo import Photo
from cloe.data_reader.reader import Reader

In [ ]:
data = like.likefinal.data
read_data = Reader(data)
read_data.compute_nz()
read_data.read_phot()
# As we currently don't have a way to load fiducial r(z), d(z) and
# H(z)s, these are currently set to match the ones from the cosmo_dic.
# This will need to be fixed once this is decided.

In [ ]:
phot_ins = Photo( cosmo_dic, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)
phot_ins_nlbias_b2 = Photo( cosmo_dic_nlbias_b2, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)
phot_ins_nlbias_nonlocal = Photo( cosmo_dic_nlbias_nonlocal, read_data.nz_dict_WL,
                      read_data.nz_dict_GC_Phot)


In [ ]:
ells= np.logspace(np.log10(10), np.log10(5000), 100)
print(type(ells))

In [ ]:
ClGC = phot_ins.Cl_GC_phot
ClGC_nlbias_b2 = phot_ins_nlbias_b2.Cl_GC_phot
ClGC_nlbias_nonlocal = phot_ins_nlbias_nonlocal.Cl_GC_phot


In [ ]:
ClGCxWL = phot_ins.Cl_cross
ClGCxWL_nlbias_b2 = phot_ins_nlbias_b2.Cl_cross
ClGCxWL_nlbias_nonlocal = phot_ins_nlbias_nonlocal.Cl_cross


### Plotting the GC_phot C_ells

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'GC photo $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:

    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]

    pl_label_0 = r'Linear bias'
    ax1.loglog(ells, ClGC(ells, iz, iz), color=colors[1], ls='-', label=pl_label_0, lw=3)

    pl_label_1 = r'Non-linear bias with $(b_1, b_2)$'
    ax1.loglog(ells, ClGC_nlbias_b2(ells, iz, iz), color=colors[2], ls='-', label=pl_label_1, lw=3)

    pl_label_2 = r'Non-linear bias with $(b_1, b_2, b_{\mathcal{G}_2}, b_{\Gamma_3})$'
    ax1.loglog(ells, ClGC_nlbias_nonlocal(ells, iz, iz), color=colors[3], ls='-', label=pl_label_2, lw=3)

    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell$ $[sr^{-1}]$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend()
plt.show()

### Plotting the WL x GC photo Cl's

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(24,18), dpi=200)
fig.suptitle(r'WL x GC photo $C^{ij}_{\ell}$')

iz_dict={1:[0,0], 3:[0,1], 5:[1,0], 10:[1,1]}

for iz in [1,3,5,10]:

    ax1 = axs[iz_dict[iz][0],iz_dict[iz][1]]

    pl_label_0 = r'Linear bias'
    ax1.semilogx(ells, ClGCxWL(ells, iz, iz), color=colors[1], ls='-', label=pl_label_0, lw=3)

    pl_label_1 = r'Non-linear bias with $(b_1, b_2)$'
    ax1.semilogx(ells, ClGCxWL_nlbias_b2(ells, iz, iz), color=colors[2], ls='-', label=pl_label_1, lw=3)

    pl_label_2 = r'Non-linear bias with $(b_1, b_2, b_{\mathcal{G}_2}, b_{\Gamma_3})$'
    ax1.semilogx(ells, ClGCxWL_nlbias_nonlocal(ells, iz, iz), color=colors[3], ls='-', label=pl_label_2, lw=3)

    ax1.set_xlabel(r'$\ell$', fontsize=26)
    ax1.set_ylabel(r'$C_\ell$ $[sr^{-1}]$', fontsize=26)
    ax1.set_title('bin i,j={:d},{:d}'.format(iz,iz))
    ax1.legend()
plt.show()

# Timing NL bias

In [ ]:
# First copy the info dict
info_speed = copy.deepcopy(info)

# We measure the time it takes for a posterior evaluation. Arbitrarily, we choose to sample ns.
info_speed['params']['ns'] = {'prior':{'min':0.95, 'max':0.97}}

# We start with NL_flag_phot_bias = 0
info_speed['likelihood']['Euclid']['NL_flag_phot_bias'] = 0

model_speed = get_model(info_speed)

# Evaluate posterior once to ensure that everything is loaded before measuring time
point = dict(zip(model_speed.parameterization.sampled_params(),
                 model_speed.prior.sample(ignore_external=True)[0]))
logposterior = model_speed.logposterior(point)

# Evalute posterior n_eval times and average

n_eval = 10

t1 = time.time()

for i in range(n_eval):
    point = dict(zip(model_speed.parameterization.sampled_params(),
                     model_speed.prior.sample(ignore_external=True)[0]))

    logposterior = model_speed.logposterior(point)

t2 = time.time()

t_ave = (t2 - t1) / n_eval

print("\nAverage time for posterior evaluation for NL_flag_phot_bias =",
      info_speed['likelihood']['Euclid']['NL_flag_phot_bias'],
      "is", t_ave, "seconds.")

In [ ]:
# First copy the info dict
info_speed = copy.deepcopy(info)

# We measure the time it takes for a posterior evaluation. Arbitrarily, we choose to sample ns.
info_speed['params']['ns'] = {'prior':{'min':0.95, 'max':0.97}}

# We start with NL_flag_phot_bias = 1
info_speed['likelihood']['Euclid']['NL_flag_phot_bias'] = 1

model_speed = get_model(info_speed)

# Evaluate posterior once to ensure that everything is loaded before measuring time
point = dict(zip(model_speed.parameterization.sampled_params(),
                 model_speed.prior.sample(ignore_external=True)[0]))
logposterior = model_speed.logposterior(point)

# Evalute posterior n_eval times and average

n_eval = 10

t1 = time.time()

for i in range(n_eval):
    point = dict(zip(model_speed.parameterization.sampled_params(),
                     model_speed.prior.sample(ignore_external=True)[0]))

    logposterior = model_speed.logposterior(point)

t2 = time.time()

t_ave = (t2 - t1) / n_eval

print("\nAverage time for posterior evaluation for NL_flag_phot_bias =",
      info_speed['likelihood']['Euclid']['NL_flag_phot_bias'],
      "is", t_ave, "seconds.")